# 10 Date transformation


In [1]:
import pandas as pd

In [3]:
from datetime import date

pdf = pd.DataFrame({'date':
['01-01-2017',
'04-12-2008',
'23-06-1988',
'25-08-1999',
'20-02-1993',
]})

#Transform string to date
pdf['date'] = pd.to_datetime(pdf.date, format="%d-%m-%Y")

In [4]:
pdf.head()

,date
0,2017-01-01
1,2008-12-04
2,1988-06-23
3,1999-08-25
4,1993-02-20


# 10.1 Transform date with pandas

## 10.1.1 Explode date

Extracting the parts of the date into different columns: Year, month, day

In [5]:
res=pdf.copy()
#Extracting Year
res['year'] = res['date'].dt.year

res.head()


,date,year
0,2017-01-01,2017
1,2008-12-04,2008
2,1988-06-23,1988
3,1999-08-25,1999
4,1993-02-20,1993


In [6]:
#Extracting Month
res['month'] = res['date'].dt.month

res.head()

,date,year,month
0,2017-01-01,2017,1
1,2008-12-04,2008,12
2,1988-06-23,1988,6
3,1999-08-25,1999,8
4,1993-02-20,1993,2


In [7]:
#Extracting day
res['day'] = res['date'].dt.day

res.head()

,date,year,month,day
0,2017-01-01,2017,1,1
1,2008-12-04,2008,12,4
2,1988-06-23,1988,6,23
3,1999-08-25,1999,8,25
4,1993-02-20,1993,2,20


## 10.1.2 Calculate the time period

Extracting the time period between the current date and columns in terms of years, months, days, etc.

In [8]:
period=pdf.copy()

#Extracting passed years since the date
period['passed_years'] = date.today().year - period['date'].dt.year
period.head()

,date,passed_years
0,2017-01-01,5
1,2008-12-04,14
2,1988-06-23,34
3,1999-08-25,23
4,1993-02-20,29


In [9]:
#Extracting passed months since the date
period['passed_months'] = (date.today().year - period['date'].dt.year) * 12 + date.today().month - period['date'].dt.month

period.head()

,date,passed_years,passed_months
0,2017-01-01,5,66
1,2008-12-04,14,163
2,1988-06-23,34,409
3,1999-08-25,23,275
4,1993-02-20,29,353


## 10.1.3 Get special day marks

Extracting some specific features from the date: Name of the weekday, Weekend or not, holiday or not, etc.

In [10]:
special_day=pdf.copy()


#Extracting the weekday name of the date
special_day['day_name'] = special_day['date'].dt.day_name()

special_day.head()

,date,day_name
0,2017-01-01,Sunday
1,2008-12-04,Thursday
2,1988-06-23,Thursday
3,1999-08-25,Wednesday
4,1993-02-20,Saturday


# 10.2 Transform date with spark

Prepare spark context and data frame


In [42]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col, to_date, year,month,dayofmonth,current_date,datediff,round,floor,ceil,months_between,dayofweek
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [2]:
local = True
if local:
    spark = SparkSession.builder \
        .master("local[4]") \
        .appName("Feature_scaling") \
        .getOrCreate()
else:
    spark = SparkSession.builder \
        .master("k8s://https://kubernetes.default.svc:443") \
        .appName("Feature_scaling") \
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master") \
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
        .config("spark.executor.instances", "4") \
        .config("spark.executor.memory", "2g") \
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
        .getOrCreate()

# make the large dataframe show pretty
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

22/07/22 10:48:48 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.184.146 instead (on interface ens33)
22/07/22 10:48:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/22 10:48:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [23]:
data=[('01-01-2017',),('04-12-2008',),('23-06-1988',),('25-08-1999',),('20-02-1993',)]
schema=StructType([StructField("date_str",StringType(),True)])

df = spark.createDataFrame(data,schema)

In [24]:
df.show()

+----------+
|  date_str|
+----------+
|01-01-2017|
|04-12-2008|
|23-06-1988|
|25-08-1999|
|20-02-1993|
+----------+



In [25]:
df.printSchema()

root
 |-- date_str: string (nullable = true)



# Convert string column to date column

In [26]:
df=df.withColumn("date",to_date(col("date_str"),"dd-mm-yyyy"))

df.show()
df.printSchema()

+----------+----------+
|  date_str|      date|
+----------+----------+
|01-01-2017|2017-01-01|
|04-12-2008|2008-01-04|
|23-06-1988|1988-01-23|
|25-08-1999|1999-01-25|
|20-02-1993|1993-01-20|
+----------+----------+

root
 |-- date_str: string (nullable = true)
 |-- date: date (nullable = true)



## 10.2.1 Explode date

Extracting the parts of the date into different columns: Year, month, day

In spark, to get the day, we have three options:
- dayOfYear
- dayOfMonth
- dayOfWeek

In [27]:
date_col_name="date"
date_extraction=df.withColumn("Year",year(col(date_col_name)))\
    .withColumn("Month",month(col(date_col_name)))\
    .withColumn("day",dayofmonth(col(date_col_name)))

date_extraction.show()
date_extraction.printSchema()

+----------+----------+----+-----+---+
|  date_str|      date|Year|Month|day|
+----------+----------+----+-----+---+
|01-01-2017|2017-01-01|2017|    1|  1|
|04-12-2008|2008-01-04|2008|    1|  4|
|23-06-1988|1988-01-23|1988|    1| 23|
|25-08-1999|1999-01-25|1999|    1| 25|
|20-02-1993|1993-01-20|1993|    1| 20|
+----------+----------+----+-----+---+

root
 |-- date_str: string (nullable = true)
 |-- date: date (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- day: integer (nullable = true)



## 10.2.2 Calculate the time period

Extracting the time period between the current date and columns in terms of years, months, days, etc.

In spark the `datediff` function returns the diff of two dates in days. So for the difference in year, and month, we need to calculate and round it with various function
(e.g. round, floor, ceil) based on your requirements.

for the month diff, we can notice the function months_between is more accurate than my perso calculation

In [49]:
date_period=df.withColumn("current_date",current_date())\
    .withColumn("passed_days",datediff(col("current_date"),col("date")))\
    .withColumn("passed_month",months_between(col("current_date"),col("date")))\
    .withColumn("passed_year",ceil(col("passed_days")/365))\
    .withColumn("passed_month_calc",ceil(col("passed_days")/30))

In [50]:
date_period.show()

+----------+----------+------------+-----------+------------+-----------+-----------------+
|  date_str|      date|current_date|passed_days|passed_month|passed_year|passed_month_calc|
+----------+----------+------------+-----------+------------+-----------+-----------------+
|01-01-2017|2017-01-01|  2022-07-22|       2028| 66.67741935|          6|               68|
|04-12-2008|2008-01-04|  2022-07-22|       5313|174.58064516|         15|              178|
|23-06-1988|1988-01-23|  2022-07-22|      12599|413.96774194|         35|              420|
|25-08-1999|1999-01-25|  2022-07-22|       8579|281.90322581|         24|              286|
|20-02-1993|1993-01-20|  2022-07-22|      10775|354.06451613|         30|              360|
+----------+----------+------------+-----------+------------+-----------+-----------------+



## 10.2.3 Get special day marks

Extracting some specific features from the date: Name of the weekday, Weekend or not, holiday or not, etc.

Unlike pandas, the dayofweek function does not return "Monday", etc. It returns an Integer (between 1, 7)

In [51]:
date_spec=df.withColumn("day_name",dayofweek("date"))

In [52]:
date_spec.show()

+----------+----------+--------+
|  date_str|      date|day_name|
+----------+----------+--------+
|01-01-2017|2017-01-01|       1|
|04-12-2008|2008-01-04|       6|
|23-06-1988|1988-01-23|       7|
|25-08-1999|1999-01-25|       2|
|20-02-1993|1993-01-20|       4|
+----------+----------+--------+



In [53]:
date_spec.printSchema()

root
 |-- date_str: string (nullable = true)
 |-- date: date (nullable = true)
 |-- day_name: integer (nullable = true)

